# PHP 2022

## 5. PHP Data Objects

### 5.1 Databases

#### MySQL

First, let us list currently running Docker containers:

In [1]:
! docker ps

CONTAINER ID   IMAGE                    COMMAND                  CREATED          STATUS          PORTS     NAMES
f18277bc8ee1   ddgg/php_2022:notes_v1   "/bin/sh -c '. /home…"   11 minutes ago   Up 11 minutes             cranky_lamport


There should be only single container for this notebook.

Now we can start the ```mysql/mysql-server:8.0``` image:

In [8]:
! docker run \
    --name=mysql \
    --net=host \
    --rm \
    --env MYSQL_ROOT_PASSWORD=root123 \
    --env MYSQL_ROOT_HOST=% \
    --env MYSQL_DATABASE=test \
    --env MYSQL_USER=test \
    --env MYSQL_PASSWORD=test123 \
    -d mysql/mysql-server:8.0

c70dcecf488ee8bb32b6248b15554b367767f0963607f77551ee04a089c4fdc1


The databse container named ```mysql``` (```--name=mysql```) will run on the same network as this notebbok (```--net=host```). The contents of the database will be removed after container is stopped (```--rm```). The root password is set to ```root123``` (```--env MYSQL_ROOT_PASSWORD=root123```) and root connections are accepted from any host (```--env MYSQL_ROOT_HOST=%```). Clients will use the ```test``` databse (```--env MYSQL_DATABASE=test```) and ```test``` user (```--env MYSQL_USER=test```) with password ```test123``` (```--env MYSQL_PASSWORD=test123```).

The ```mysql``` container should now be visible:

In [4]:
! docker ps

CONTAINER ID   IMAGE                    COMMAND                  CREATED          STATUS                   PORTS     NAMES
794d5bf16aac   mysql/mysql-server:8.0   "/entrypoint.sh mysq…"   5 minutes ago    Up 5 minutes (healthy)             mysql
f18277bc8ee1   ddgg/php_2022:notes_v1   "/bin/sh -c '. /home…"   17 minutes ago   Up 17 minutes                      cranky_lamport


It takse some tmie to start the database:

In [5]:
! while ! timeout 1 bash -c "echo > /dev/tcp/localhost/3306" 2> /dev/null; \
  do \
    echo "Waiting for MySQL..."; \
    sleep 1; \
  done; \
  echo "Done."

Done.


When done, the port ```3306``` should be opened.

If something goes wrong, logs can be checked using:

In [5]:
! docker logs mysql

[Entrypoint] MySQL Docker Image 8.0.32-1.2.11-server
[Entrypoint] Initializing database
2023-03-30T08:19:20.104333Z 0 [Warning] [MY-011068] [Server] The syntax '--skip-host-cache' is deprecated and will be removed in a future release. Please use SET GLOBAL host_cache_size=0 instead.
2023-03-30T08:19:20.104397Z 0 [System] [MY-013169] [Server] /usr/sbin/mysqld (mysqld 8.0.32) initializing of server in progress as process 17
2023-03-30T08:19:20.117580Z 1 [System] [MY-013576] [InnoDB] InnoDB initialization has started.
2023-03-30T08:19:22.825639Z 1 [System] [MY-013577] [InnoDB] InnoDB initialization has ended.
2023-03-30T08:19:26.678869Z 6 [Warning] [MY-010453] [Server] root@localhost is created with an empty password ! Please consider switching off the --initialize-insecure option.
[Entrypoint] Database initialized
2023-03-30T08:19:34.859547Z 0 [Warning] [MY-011068] [Server] The syntax '--skip-host-cache' is deprecated and will be removed in a future release. Please use SET GLOBAL host_ca

Open bash session and connect to server using docker mysql client image:

```bash
mysql -h127.0.0.1 -utest -p
```

You have to type ```test123``` as password.

Example commands that can be excued in MySQL:

```sql
-- Show available databses
show databases;

-- Select database you are interested in
use test;

-- Show available tables
show tables;

-- Create new table
CREATE TABLE test (id INT PRIMARY KEY, value TEXT);

-- Show table format
DESCRIBE test;

-- Insert some values
INSERT INTO test VALUES (1, "PL"), (2, "EN");

-- Select all rows
SELECT * FROM test;

-- Drop table
DROP TABLE test;

-- Finish session
exit
```

To stop the database execute:

In [7]:
! docker container stop mysql

Error response from daemon: No such container: mysql


### SQLite

Open bash session and open the databse using ```sqlite3``` command:

```bash
sqlite3 /tmp/sqlite.db 
```

Example commands that can be excued in SQLite:

```sql
-- Non-SQL command start with dot '.' - for help type:
.help

-- Show tables
.tables

-- Create dummy table (SQL)
-- HINT: If you get 'readonly database' error check file write permissinons
CREATE TABLE dummy (id INTEGER PRIMARY KEY, data TEXT NOT NULL);

-- Show tables
.tables

-- Show command that was used to create given table
.schema dummy

-- Insert some data into table
INSERT INTO dummy VALUES (1, "Foo"), (2, "Bar");

-- Select values from table
SELECT * FROM dummy;

-- If you do not like default ouput format
.headers on
.mode column

-- Select values from table
SELECT * FROM dummy;

-- Drop table
DROP TABLE dummy;

-- Exit from database
.quit
```

Clenaup:

In [ ]:
! rm -f /tmp/sqlite.db 